# Second Parser: Lines and Breaks
by Niek Veldhuis
UC Berkeley

July 2017


# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).
* clean up Notebook


# Introduction

The two main differences between `First_JSON_parser.ipynb` and the current notebook are

- the ability to parse an entire corpus
- recognizing lines
- including breaks (as in "3 lines broken").

Although these features somewhat complicate the code, the basic techniques used are the same.

The resulting data file may include various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes

This notebook is written for **Python 3.5** with **Pandas 0.19** and **requests 2.12.4**.


## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [ ]:
import pandas as pd   
import requests
import zipfile
import tqdm
import numpy as np

# Input List of Text IDs or a project abbreviation
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

Alternatively, one may enter the name (abbreviation) of a project or sub-project in [ORACC](http://oracc.org) and pull all the lemmatized data from that project. Note that the script will not automatically pull data from subprojects, they have to be requested separately. Examples:
* saao/saa01
* aemw/amarna
* rimanum

In [ ]:
name = input('Filename or project abbreviation: ')

In [ ]:
if name[-4:] == '.txt':
    textids = 'text_ids/' + name
    with open(textids, 'r') as f:
        pqxnos = f.readlines()
    pqxnos = [x.strip() for x in pqxnos]  # strip spaces left and right
    pqxnos = [x for x in pqxnos if not x == ""] # strip empty lines
#    pqxnos = [x[-7:].upper() for x in pqxnos]
    projects = [x[:-8].lower() for x in pqxnos]
    projects = list(set(projects))
else:
    name = name.strip().lower()
    projects = [name]
    url = "http://oracc.org/" + name + "/corpus.json"
    r = requests.get(url)
    corpus = r.json()
    pqxnos = list(corpus["members"].keys())
    pqxnos = [name + '/' + no for no in pqxnos]

In [ ]:
projects

## 1.2 Create Download Directory and JSON directory
For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist)

In [ ]:
import errno
import os
try:
    os.mkdir('jsonzip')
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass
try:
    os.mkdir('json')
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

## 1.3 Download `json.zip`
For each project from which files are to be processed download the entire project (all the json files) in `https://github.com/oracc/json`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

Although downloading the entire zip file is time consuming, it will make processing the individual files much more efficient and the code is less likely to break due to interruption in connectivity.

## Note:
It may be better to download the `zip` file from [ORACC](http://oracc.org), where it is available as `http://oracc.org/[PROJECT]/json.zip`. This version is updated when a project is updated. Right now the file seems to be not accessible.

In [ ]:
CHUNK = 16 * 1024
for project in tqdm.tqdm(projects):
    project = project.replace('/', '-')
    url = "https://raw.github.com/oracc/json/master/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    print("Downloading " + url + " saving as " + file)
    r = requests.get(url)
    with open(file, 'wb') as f:
        for c in r.iter_content(chunk_size=CHUNK):
            f.write(c)

## 1.4 Extract JSON files from `json.zip`
Extract the texts listed in the list of text IDs from the `json.zip`. All files are extracted to a directory called `data/[PROJECT]/json/corpusjson` (for instance `data/dcclt/json/corpusjson`). If the file belongs to a subproject the directory is called `data/[PROJECT]/[SUBPROJECT]/json/corpusjson`. 

In [ ]:
target_dir = 'json'
for no in tqdm.tqdm(pqxnos):
    project = no[:-8].lower()
    pno = no[-7:].upper()
    zip_file = "jsonzip/" + project.replace('/', '-') + ".zip"
    with zipfile.ZipFile(zip_file,"r") as zip_ref:
        file = project + '/corpusjson/' + pno + '.json'
        try:
            zip_ref.extract(file, target_dir)
        except:
            print(no + ' is not available')

## 1.5 Parse JSON files
The `parsejson()` function is essentially identical with the that function in `First_JSON_parser.ipynb`, but it fetches more data. The field `word_id` consists of three parts, namely a text ID, line ID, and word ID, in the format `Q000039.76.2` meaning: the second word in line 76 of text object `Q000039`. Note that `76` is not a line number strictly speaking but an object reference within the text object. Things like horizontal rulings, columns, and breaks also get object references. The `word_id` field allows us to put lines together in the proper order.

The field `label` is a human-legible label that refers a line or another part of the text; it may look like `o i 23` (obverse column 1 line 23) or `r v 23'` (reverse column 5 line 23 prime). The `label` field is used in online [ORACC](http://oracc.org) editions to indicate line numbers.

The fields `extent`, `scope`, and `state` give metatextual data about the condition of the object; they capture the number of broken lines or columns and similar information. 



In [ ]:
def parsejson(text, lemm_l = None):
    dollar_keys = ["extent", "scope","state"]
    if lemm_l == None:
        lemm_l = []
    for dictionary in text["cdl"]:
        if "cdl" in dictionary: 
            parsejson(dictionary, lemm_l)
        if "label" in dictionary:
            label = dictionary['label']
        if "f" in dictionary:
            lemma = dictionary["f"]
            lemma["id_word"] = dictionary["ref"]
            lemma['label'] = label
            lemm_l.append(lemma)
        if "strict" in dictionary and dictionary["strict"] == "1":
            lemma = {key: dictionary[key] for key in dollar_keys}
            lemma["id_word"] = dictionary["ref"] + ".0"
            lemm_l.append(lemma)
    return lemm_l

## 1.6 Call the Parser Function for Each Textid

In [ ]:
word_l = []
for id_text in tqdm.tqdm(pqxnos):
    try:
        word_l.extend(oraccjsonparser(id_text))
    except:
        print(no + ' is not available or not complete')

## 1.7 Transform the Data into a DataFrame
The word_l list is transformed into a Pandas dataframe for further manipulation.

For various reasons not all JSON files will have all data types that potentially exist in an [ORACC](http://oracc.org) signature. Only Sumerian words have a `base`, so if your data set has no Sumerian, this column will not exist in the DataFrame.  If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. Since such fields are referenced in the code below (sections 2-4) the next cell will check for the existence of each column and create an empty column if necessary.

In [ ]:
words = pd.DataFrame(word_l)
fields = ['base', 'cf', 'cont', 'epos', 'extent', 'form', 'gw', 'id_word',
          'label', 'lang', 'morph', 'norm', 'norm0', 'pos', 'scope', 'sense', 'sig']
for field in fields:
    if not field in words.columns:
        words[field] = ''
words = words.fillna('') # replace Missing Values by empty string
words.head(100)

## 1.8 Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [ ]:
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* frag (transliteration; including flags)
* gdl_utf8 (cuneiform)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* sig (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# 2. Manipulate for SNA
The columns of the `words` DataFrame may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will select proper nouns only and create two `.csv` files (`edges.csv` and `nodes.csv`) that may be ingested by Social Network Analysis (SNA) software. The column names follow the conventions used in [Gephi](https://gephi.org/).

## 2.1 Select Proper Nouns
First list all Part of Speech tags currently available in the corpus.

In [ ]:
pos = list(set(words['pos']))
pos

Then list the tags that are relevant in the list `pos` and use that list to select the rows of the DataFrame that contain proper nouns.  

In [ ]:
pos = ['CN', 'DN', 'EN', 'FN', 'GN', 'PN', 'NN', 'RN', 'SN', 'TN', 'WN'] # what is 'NN'?
proper_nouns = words.loc[words['pos'].isin(pos)].reset_index(drop=True)
proper_nouns.head()

## 2.2 Keep  Norm, Pos, and id_text
Now select the relevant columns.

In [ ]:
proper_nouns = proper_nouns[['norm', 'pos', 'id_text']].drop_duplicates()
proper_nouns = proper_nouns[proper_nouns['norm'] != ''].reset_index(drop=True)
proper_nouns.head()

## 2.3 Create Edge List
The edge list contains the columns `source` and `target` and combines all proper nouns that appear in a single text as source-target pairs. All edges are considered `undirected`.

In [ ]:
edges = []
for i in tqdm.tqdm(range(len(proper_nouns))):
    for j in range(i+1, len(proper_nouns)):
        if proper_nouns['id_text'][i] == proper_nouns['id_text'][j]:
            edge = [proper_nouns['norm'][i], proper_nouns['norm'][j]]
            edges.append(edge)
        else:
            break
edges_df = pd.DataFrame(edges)
edges_df.columns = ['source', 'target']
edges_df['type'] = 'undirected'
edges_df.head()

In [ ]:
with open("output/edges.csv", 'w') as f:
    edges_df.to_csv(f, index=False)

## 2.4 Create Node List
pn_set contains the unique proper nouns in the entire corpus. This become the node list in Gephi

In [ ]:
pn_set = proper_nouns[['norm', 'pos']].drop_duplicates() # Assur DN and Assur GN are not considered duplicates!
pn_set.columns = ['Id', 'Type']
pn_set['Label'] = pn_set['Id']
pn_set.head()

In [ ]:
with open("output/nodes.csv", 'w') as f:
    pn_set.to_csv(f, index=False)

# 3. Manipuate for Analysis on Line level (e.g. phylogenetics)
For analyses that use a line as unit of analysis (e.g. lines in lexical texts as analyzed in the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project) one may need to create lemmas and combine these into lines by using the `id_line` variable.

## 3.1 Create Lemmas and Adjust Bases
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string.

For Sumerian projects each lemmatized word has a `base` (the word without morphology). For non-lemmatized words a place-holder base is created that consists of the transliteration of the word. If you are not working with Sumerian data.

In [ ]:
words["lemma"] = words.apply(lambda r: (r["cf"] + '[' + r["gw"] + ']' + r["pos"]) 
                            if r["cf"] != '' else r['form'] + '[NA]NA', axis=1)
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = words.apply(lambda r: r["base"] if r["base"] != '' or r['label'] == '' else r['form'], axis=1)

## 3.2 Group by Line
In the `words` dataframe each word has a separate row. In order into change this to a line-by-line representation we use the Pandas `.groupby` function, using the `id_line` and `label` fields as arguments (`id_line` has an abstract number that indicates the sequence of lines in a text object; `label` is a human-readable line number in the format `o ii 3`: obverse column 2, line 3). The fields that are aggregated are `lemma`, `base`, `extent`, and `scope`. The fields `extent` and `scope` represent data on the number of broken lines. If you work with Akkadian data you want to leave out the field `base`.

In [ ]:
words['id_line'] = [wordid[:wordid.rfind('.')+1] for wordid in words['id_word']]

In [ ]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
lines        

Note that `id_line` is a string variable and therefore does not give the lines in the right order. We should split `id_line` into two variables: `id_text` (the first 7 characters; we lost the old `id_text` column in the `.groupby` function above) and a new `line` variable, which is a number. 

In [ ]:
lines['id_text'] = lines['id_line'].str[:7] # id_text was lost in the grouping above and is recreated
lines['line'] = [line[line.rfind('.')+1:] for line in lines['id_line']] #create a line number for sorting
lines['line'] = [x.replace('l', '') for x in lines['line']]
lines['line'] = [int(x) if not x == '' else np.nan for x in lines['line']]
lines = lines.sort_values(['id_text', 'line']).reset_index(drop=True)
lines.head(100)

Note that the new `line` field is not a line number in the traditional sense of the word (this is `label`) but a number used to organize lines in the appropriate order.

## 3.3 Save in CSV Format

In [ ]:
filename = filename[:-4]
with open('output/' + filename + '.csv', 'w') as w:
    lines.to_csv(w, encoding='utf8')

# 4 Manipulate for Document-level Analysis
For analyses that use documents in a Document Term Matrix or otherwise a similar type of manipulation is needed. This output may be used in Word2vec, in Topic Modeling and in other types of algorithms. First lemmas and bases are dealt with in the same way as above, section 3.1.

## 4.1 Create Lemmas and Adjust Bases
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Suerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string.

For Sumerian projects each lemmatized word has a `base` (the word without morphology). For non-lemmatized words a place-holder base is created that consists of the transliteration of the word. If you are not working with Sumerian data.

In [ ]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]

# 4.1 Group by Document
In order to group by Document we use the field `id_text` and aggregate `lemma` and `base`.

In [ ]:
documents = words.groupby(words['id_text']).agg({
        'lemma': ' '.join,
        'base': ' '.join,
    }).reset_index()
documents

## 3.4 Save in CSV Format

In [ ]:
filename = filename[:-4]
with open('output/' + filename + '.csv', 'w') as w:
    lines.to_csv(w, encoding='utf8')

## 3.5 Tokenizing
Since lemmas do not contain spaces (see above, section 1.8) tokenizing is extremely easy and basically consists of splitting on spaces. Tokenized data are used in Topic Modeling, Word2vec, etc. Tokenizing is also necessary for making a Document Term Matrix.

In [ ]:
documents['tokens'] = documents['lemma'].str.split()
documents